### Import modules

In [23]:
import torch as t
import torch.nn as nn #For neural networks

### Defining a custom neural network

In [24]:
class custom_nn (nn.Module):
    def __init__(self, n_inputs, n_hidden_neurons, n_outputs):
        super(custom_nn, self).__init__()
        self.model = nn.Sequential(nn.Linear(n_inputs, n_hidden_neurons),
                                   nn.ReLU(),
                                   nn.Linear(n_hidden_neurons, n_outputs),
                                   nn.Tanh()
                                   )
    
    def forward(self, x):
        return self.model(x)

### Training a neural network

In [25]:
%%time
ninputs = 10 #No of inputs
nhidden = 5 #No of hidden neurons
noutputs = 10 #No of outputs
nexamples = 15 #No of examples to given to the neural network

x = t.randn(nexamples, ninputs)
y = x #Output is same as input - an autoencoder
print(x.shape)
print(y.shape)
  
nn1 = custom_nn(ninputs, nhidden, noutputs) #Create an object (nn1) of custom_nn class

loss_fn = nn.MSELoss() #Loss function
optimizer = t.optim.SGD(nn1.parameters(), lr = 0.01) #Defining the optimizer - SGD
epochs = 100000

print("Wait - training the model")
#Gradient Descent Algorithm
for i in range(epochs):
    ypred = nn1(x) #Calculate predicted value of the model
    loss = loss_fn(ypred, y) #Apply the loss function (MSE) to calculate MSE
    if ((i+1) % 10000 == 0):
        print("Epoch : ", i+1, "Loss = ", loss.item()) #Prints loss after each 10,000 epochs
    loss.backward() #Backward - Backward propagation
    optimizer.step() #Update all parameters
    optimizer.zero_grad() #Set gradients of all parameters to zero before starting the next epoch
    
print("Loss = ", loss.item()) #Prints the final loss

torch.Size([15, 10])
torch.Size([15, 10])
Wait - training the model
Epoch :  10000 Loss =  0.3378535211086273
Epoch :  20000 Loss =  0.3123736083507538
Epoch :  30000 Loss =  0.3037499189376831
Epoch :  40000 Loss =  0.2975807785987854
Epoch :  50000 Loss =  0.29111090302467346
Epoch :  60000 Loss =  0.2869543433189392
Epoch :  70000 Loss =  0.2834678292274475
Epoch :  80000 Loss =  0.2808316648006439
Epoch :  90000 Loss =  0.2790692150592804
Epoch :  100000 Loss =  0.27769023180007935
Loss =  0.27769023180007935
CPU times: user 2min 37s, sys: 3.27 s, total: 2min 40s
Wall time: 26.7 s


### Predicting output for unseen inputs

In [26]:
x_new = t.randn(1, ninputs) #New inputs
y_new = nn1(x_new) #Predicting outputs using the trained model
print("x_new = ", x_new)
print("y_new = ", y_new) 

x_new =  tensor([[-0.4099, -1.2812,  0.6133, -0.4557,  0.8936, -1.0523, -0.5323, -0.3767,
         -0.7701, -0.3890]])
y_new =  tensor([[ 1.0000, -0.9952,  0.7033,  0.9185,  0.9092, -0.9557,  1.0000,  0.1148,
          0.9593, -0.0244]], grad_fn=<TanhBackward>)


### Training the neural network and data on a GPU if it is available

In [27]:
%%time

if t.cuda.is_available():
    device = t.device("cuda:0") #If a GPU is availble, set device as first GPU (0 means first GPU)
    print("Running on the GPU = ", device)
else:
    device = t.device("cpu") #Else set the device as CPU
    print("Running on the CPU = ", device)

ninputs = 10 #No of inputs
nhidden = 5 #No of hidden neurons
noutputs = 10 #No of outputs
nexamples = 15 #No of examples to given to the neural network

x = t.randn(nexamples, ninputs, device=device)   #IMPORTANT - x is now on GPU if a GPU is available
y = x #Output is same as input - an autoencoder  #IMPORTANT - y too is now on GPU if a GPU is available

#IMPORTANT - be cautious if data is too large and if GPU memory is not fit enough for the data 

print(x.shape)
print(y.shape)
  
nn1 = custom_nn(ninputs, nhidden, noutputs) #Create an object (nn1) of custom_nn class
nn1 = nn1.to(device) #IMPORTANT - Neural network nn1 is passed to GPU if a GPU is available

loss_fn = nn.MSELoss() #Loss function
optimizer = t.optim.SGD(nn1.parameters(), lr = 0.01) #Defining the optimizer - SGD
epochs = 100000

print("Wait - training the model")
#Gradient Descent Algorithm
for i in range(epochs):
    ypred = nn1(x) #Calculate predicted value of the model
    loss = loss_fn(ypred, y) #Apply the loss function (MSE) to calculate MSE
    if ((i+1) % 10000 == 0):
        print("Epoch : ", i+1, "Loss = ", loss.item()) #Prints loss after each 10,000 epochs
    loss.backward() #Backward - Backward propagation
    optimizer.step() #Update all parameters
    optimizer.zero_grad() #Set gradients of all parameters to zero before starting the next epoch
    
print("Loss = ", loss.item()) #Prints the final loss

Running on the GPU =  cuda:0
torch.Size([15, 10])
torch.Size([15, 10])
Wait - training the model
Epoch :  10000 Loss =  0.26431289315223694
Epoch :  20000 Loss =  0.2288837730884552
Epoch :  30000 Loss =  0.21779102087020874
Epoch :  40000 Loss =  0.21157768368721008
Epoch :  50000 Loss =  0.2073155641555786
Epoch :  60000 Loss =  0.20453552901744843
Epoch :  70000 Loss =  0.2024114578962326
Epoch :  80000 Loss =  0.20072896778583527
Epoch :  90000 Loss =  0.1993687003850937
Epoch :  100000 Loss =  0.19824984669685364
Loss =  0.19824984669685364
CPU times: user 54.2 s, sys: 1.65 s, total: 55.8 s
Wall time: 55 s
